In [ ]:
import pandas as pd
import os
import pandas as pd

# กำหนดคอลัมน์ตามตาราง gis_vector (ยกเว้น embedding)
columns = [
    "ca_number",
    "pea_number",
    "customer_name",
    "customer_address",
    "office_code",
    "lat",
    "long",
    "billing_month"
]

def main(input_file: str):
    output_file = input_file.replace(".TXT", ".csv")    # ไฟล์ CSV ที่ต้องการบันทึก

    # อ่านไฟล์ .TXT
    df = pd.read_csv(
        input_file,
        sep="|",
        header=None,
        dtype=str,
        encoding="cp874",   # ใช้ cp874 (windows-874) ถ้าอ่านไม่ได้ลอง "tis-620"
        on_bad_lines="skip",
        engine="python"
    )

    # เลือกเฉพาะคอลัมน์ที่ต้องใช้ (ตามลำดับ)
    df = df.iloc[:, [1, 0, 9, 10, 4, 2, 3, 8]]
    df.columns = columns

    # ลบค่าว่าง
    df = df.dropna()

    # ตัดช่องว่างซ้ายขวาออกทุก field
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    # แปลง lat/long เป็นตัวเลข
    df["lat"] = pd.to_numeric(df["lat"], errors="coerce")
    df["long"] = pd.to_numeric(df["long"], errors="coerce")

    # ลบแถวที่ไม่มี ca_number (Primary Key)
    df = df[df["ca_number"].notna() & (df["ca_number"] != "")]

    # ลบแถวที่มี ca_number ซ้ํา
    df_cleaned = df.drop_duplicates(subset="ca_number", keep="first").copy()  # <- add .copy()

    # บันทึกเป็น CSV
    df_cleaned.to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"✅ Data cleaned and saved to {output_file}")

if __name__ == "__main__":
    folder_name = 'gis_data'
    print(os.listdir(folder_name))
    for file in os.listdir(folder_name):
        if file.endswith(".TXT"):
            main(os.path.join(folder_name, file))
            # break

# DONE

---

In [ ]:
import pandas as pd

df = pd.read_csv('gis_data/GIS_A_202507.csv')

df.head()

In [ ]:
df.count()

# สมมติ df คือ DataFrame ของคุณ
full_count = len(df)  # 2054512 ตามตัวอย่าง

for col in df.columns:
    non_missing = df[col].notna().sum()  # จำนวนค่าที่ไม่ว่าง
    missing = full_count - non_missing   # จำนวนค่าที่ขาด
    print(f"{col:20} Non-missing: {non_missing:10}  Missing: {missing:10}")